In [1]:
import os
import sys

import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F  
import matplotlib 
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
import torch.utils.data
from torch.utils.data import DataLoader

In [2]:
EPSILON = 0.1       # Max perturbation (for L∞ PGD)
ALPHA = 0.01         # Step size per iteration
ATTACK_ITERATIONS = 40
TARGET_LABEL = 1     # Example target label for the targeted attack

# System/Model parameters
sys.path.append("/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/models")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = "/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/weights/best_model_retrained.pth"
IQ_FILE_PATH = "brandan_pluto/brandanpluto10_2m.iq"

#IQ_FILE_PATH = "/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/1m_2m_replacedPluto4/1m_2m/Pluto_0_1m_run3.iq"

from attempt2 import resnet50_1d  # Directly import from attempt2.py
num_classes = 8  # Change this if your model was trained with a different number of classes

# Initialize the model architecture
model = resnet50_1d(num_classes=num_classes).to(DEVICE)

# Load trained weights
print(f"Loading trained model weights from: {MODEL_PATH}")
state_dict = torch.load(MODEL_PATH, map_location=DEVICE)

# Load the state dictionary into the model
model.load_state_dict(state_dict)

# Set the model to evaluation mode
model.eval()



Loading trained model weights from: /home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/weights/best_model_retrained.pth


/tmp/ipykernel_3817084/1944379888.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(MODEL_PATH, map_location=DEVICE)


ResNet1D(
  (conv1): Conv1d(2, 64, kernel_size=(7,), stride=(2,), padding=(3,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck1D(
      (conv1): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv1d(64, 256, kernel_size=(1,), stride=(1,), bias=False)
      (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv1d(64, 256, kernel_size=(1,), stride=(1,), bias=False)
        (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True

In [3]:


class IQDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]

        sample = torch.from_numpy(sample).float()
        # Normalize data
        magnitude = torch.sqrt(torch.sum(sample**2, dim=1, keepdim=True))
        sample = sample / magnitude

        label_tensors = torch.tensor(label, dtype=torch.long)

        return sample, label_tensors


file_name = "May16/both.iq"
test_data_tensors = []
test_label_tensors = []
target = 1
data = np.fromfile(file_name, dtype="float32")
size_data = len(data)
print("Size of data: ", size_data)
size_half = int(size_data / 2)
real_part = data[0::2]
print("Real Length: ", len(real_part))
imag_part = data[1::2]
print("Imag Length: ", len(imag_part))


for x in range(4800, 6000):
    # Implement a 10% hop
    combined_data = np.vstack((real_part[(x+1)*500:(x+1)*500+10000], imag_part[(x+1)*500:(x+1)*500+10000]))
    array = np.zeros(combined_data.shape[1])
    array.fill(target)
    test_data_tensors.append(combined_data)
    test_label_tensors.append(target)


# Format the data
test_data_tensors = np.stack(test_data_tensors, axis=0)
test_label_tensors = np.array(test_label_tensors)

# Randomize the data
indices = np.random.permutation(len(test_data_tensors))
test_data = test_data_tensors[indices]
test_labels = test_label_tensors[indices]
print(test_data.shape)
print(test_labels.shape)

# Create the dataset
test_dataset = IQDataset(test_data, test_labels)

batch_size = 16

# Load the data
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

torch.manual_seed(7)
np.random.seed(7)

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = resnet50_1d(num_classes=8)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()
#summary(net, input_size = (1, 2), batch_size = 8)


dataiter = iter(test_loader)
images, labels = next(dataiter)

correct = 0
total = 0
conf_threshold = 0.85
fooled = 0
confidence = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(DEVICE)    # <--- ADD THIS
        labels = labels.to(DEVICE)    # <--- (optional, only if you also need labels on GPU)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        
        #for x in range(0, len(predicted)):
        #    print(predicted)
        #    print(labels)
        #    if ~(predicted[x] == labels[x]):
        #        print("Predicted: ", predicted[x])
        #        print("Label: ", labels[x])
        correct += (predicted == labels).sum().item()
        probs = torch.nn.functional.softmax(outputs, dim=1)
        conf, classes = torch.max(probs, 1)
        
        # For GAN
        
        counter = 0
        for conf_i in conf:
            # For testing normal mode
            if conf_i > conf_threshold and labels[counter] == target:
            # For generative attack
            #if (conf_i > conf_threshold):
                fooled = fooled + 1
                confidence = confidence + conf_i
            counter = counter + 1
        


### For Attack ###
fooled_acc = fooled / total * 100
if (fooled == 0):
    print("No fools")
else:
    fooled_conf = confidence / fooled * 100
    print("Average confidence: ", fooled_conf)
print('Percentage of successful fools: %d %%' % (fooled_acc))
print("Number fooled: ", fooled)
print("Number total: ", total)


Size of data:  85512720
Real Length:  42756360
Imag Length:  42756360
(1200, 2, 10000)
(1200,)


/tmp/ipykernel_3817084/4284333392.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))


KeyboardInterrupt: 